# Logistic Regression for Sentiment Analysis

Adapted from http://nbviewer.jupyter.org/github/rasbt/pattern_classification/blob/master/machine_learning/scikit-learn/outofcore_modelpersistence.ipynb

<br>
<br>

## The IMDb Movie Review Dataset

In this section, we will train a simple logistic regression model to classify movie reviews from the 50k IMDb review dataset that has been collected by Maas et. al.

> AL Maas, RE Daly, PT Pham, D Huang, AY Ng, and C Potts. Learning word vectors for sentiment analysis. In Proceedings of the 49th Annual Meeting of the Association for Computational Lin- guistics: Human Language Technologies, pages 142–150, Portland, Oregon, USA, June 2011. Association for Computational Linguistics

[Source: http://ai.stanford.edu/~amaas/data/sentiment/]

The dataset consists of 50,000 movie reviews from the original "train" and "test" subdirectories. The class labels are binary (1=positive and 0=negative) and contain 25,000 positive and 25,000 negative movie reviews, respectively.
For simplicity, I assembled the reviews in a single CSV file.


In [255]:
import pandas as pd
# if you want to download the original file:
#df = pd.read_csv('https://raw.githubusercontent.com/rasbt/pattern_classification/master/data/50k_imdb_movie_reviews.csv')
# otherwise load local file
df = pd.read_csv('shuffled_movie_data.csv')
df.tail()

,review,sentiment
49995,"OK, lets start with the best. the building. al...",0
49996,The British 'heritage film' industry is out of...,0
49997,I don't even know where to begin on this one. ...,0
49998,Richard Tyler is a little boy who is scared of...,0
49999,I waited long to watch this movie. Also becaus...,1


Let us shuffle the class labels.

In [256]:
import numpy as np
## uncomment these lines if you have dowloaded the original file:
#np.random.seed(0)
#df = df.reindex(np.random.permutation(df.index))
#df[['review', 'sentiment']].to_csv('shuffled_movie_data.csv', index=False)

<br>
<br>

## Preprocessing Text Data

Now, let us define a simple `tokenizer` that splits the text into individual word tokens. Furthermore, we will use some simple regular expression to remove HTML markup and all non-letter characters but "emoticons," convert the text to lower case, remove stopwords, and apply the Porter stemming algorithm to convert the words into their root form.

In [257]:
import numpy as np
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')
feature3 = ['no', 'not']
feature5 = ['!']
feature4 = stopwords.words('english')[:17]
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('!|(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    text = [w for w in text.split() if (w not in stop) or (w in feature3) or (w in feature4) or (w in feature5)]
    tokenized = [porter.stem(w) for w in text]
    return tokenized

Let's give it at try:

In [258]:
tokenizer('This :) me is a <a> test! :-)</br>')

['me', 'test', ':)', '!', ':)']

## Learning (SciKit)

First, we define a generator that returns the document body and the corresponding class label:

In [259]:
def stream_docs(path):
    with open(path, 'r') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

To conform that the `stream_docs` function fetches the documents as intended, let us execute the following code snippet before we implement the `get_minibatch` function:

In [260]:
next(stream_docs(path='shuffled_movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

After we confirmed that our `stream_docs` functions works, we will now implement a `get_minibatch` function to fetch a specified number (`size`) of documents:

In [261]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    for _ in range(size):
        text, label = next(doc_stream)
        docs.append(text)
        y.append(label)
    return docs, y

Next, we will make use of the "hashing trick" through scikit-learns [HashingVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html) to create a bag-of-words model of our documents. Details of the bag-of-words model for document classification can be found at  [Naive Bayes and Text Classification I - Introduction and Theory](http://arxiv.org/abs/1410.5329).

In [262]:
from sklearn.feature_extraction.text import HashingVectorizer
vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

# Excercise 1: define new features according to https://web.stanford.edu/~jurafsky/slp3/5.pdf

### Excercise 1:

In [263]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
test_subset=['20170412', 'great', 'bad', 'terrible', 'dog', 'stop', 'good', 'well']

sid = SentimentIntensityAnalyzer()
pos_word_list=[]
neu_word_list=[]
neg_word_list=[]

for word in test_subset:
    if (sid.polarity_scores(word)['compound']) >= 0.2:
        pos_word_list.append(word)
    elif (sid.polarity_scores(word)['compound']) <= -0.2:
        neg_word_list.append(word)
    else:
        neu_word_list.append(word)                

print('Positive :',pos_word_list)        
print('Neutral :',neu_word_list)    
print('Negative :',neg_word_list) 

Positive : ['great', 'good', 'well']
Neutral : ['20170412', 'dog']
Negative : ['bad', 'terrible', 'stop']


In [264]:
sid = SentimentIntensityAnalyzer()
feature4 = stopwords.words('english')[:17]

#palabrasPositivas =
#palabrasNegativas =

def featuresByText(text):
    
    vecText = text.split()
    x1 = 0
    x2 = 0
    
    x10 = 0
    x11 = 0
    
    cantNo = 0
    cantPronom = 0
    
    for word in vecText:
        #para x1 y x2
        if (sid.polarity_scores(word)['compound']) >= 0.2:
            x1 = x1 + 1
        elif (sid.polarity_scores(word)['compound']) <= -0.2:
            x2 = x2 + 1
        
       
            
        
        #para x5 y x6
        if(word=="no" or word=="not"):
            cantNo = cantNo + 1     
        
        #para x7
        for pronoun in feature4:
            if(pronoun==word):
                cantPronom = cantPronom + 1
                
    x3 = 1.0*x1/len(vecText)          
    x4 = 1.0*x2/len(vecText)          
    
    if(cantNo > 0):
        x5 = 1
    else:
        x5 = 0
    x6 = cantNo
    
    x7 = cantPronom
    
    x8 = 0 
    if (text.find('!') != -1): 
        x8 = 1 
        
    x9 = np.log( len(vecText) )
    return [x1, x2, x3, x4, x5, x6, x7, x8, x9]
        

def getFeatures(X_train):
    features = []
    for text in X_train:
        xn = featuresByText(text)
        features.append(xn)
    
    return features

datosPrueba = ["i meno hola bad bad !", "hola you weno us not ugly no carro", "caminado good!"]
print(getFeatures(datosPrueba))

[[0, 2, 0.0, 0.3333333333333333, 0, 0, 1, 1, 1.791759469228055], [0, 2, 0.0, 0.25, 1, 2, 1, 0, 2.0794415416798357], [1, 0, 0.5, 0.0, 0, 0, 0, 1, 0.6931471805599453]]


Using the [SGDClassifier]() from scikit-learn, we will can instanciate a logistic regression classifier that learns from the documents incrementally using stochastic gradient descent. 

In [265]:
#from sklearn.linear_model import SGDClassifier
#clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
#doc_stream = stream_docs(path='shuffled_movie_data.csv')
# Excercise 2: implement a Logistic Regression classifier, using regularization, according to https://web.stanford.edu/~jurafsky/slp3/5.pdf

In [266]:
#import pyprind
#pbar = pyprind.ProgBar(45)

#classes = np.array([0, 1])
#for _ in range(45):
    #X_train, y_train = get_minibatch(doc_stream, size=1000)
    #X_train = getFeatures(X_train)
    #X_train = vect.transform(X_train)
    #clf.partial_fit(X_train, y_train, classes=classes)
    

Depending on your machine, it will take about 2-3 minutes to stream the documents and learn the weights for the logistic regression model to classify "new" movie reviews. Executing the preceding code, we used the first 45,000 movie reviews to train the classifier, which means that we have 5,000 reviews left for testing:

In [267]:
#X_test, y_test = get_minibatch(doc_stream, size=5000)
#X_test = getFeatures(X_test)
#X_test = vect.transform(X_test)
#print('Accuracy: %.3f' % clf.score(X_test, y_test))

### Ejercicio 2

In [268]:
def sigmoid(z):
    return 1 / (1+np.exp(-z))

In [269]:
doc_stream = stream_docs(path='shuffled_movie_data.csv')
#Obteniendo data para entrenar
X_train, Y_train = get_minibatch(doc_stream, size=45000)
X_train = getFeatures(X_train) 

In [270]:
X_test, Y_test = get_minibatch(doc_stream, size=5000)
X_test = getFeatures(X_test) 

In [271]:
def lrCostFunction(theta_t, X_t, y_t, lambda_t):
    m = len(y_t)
    J = (-1/m) * (y_t.T @ np.log(sigmoid(X_t @ theta_t)) + (1 - y_t.T) @ np.log(1 - sigmoid(X_t @ theta_t)))
    reg = (lambda_t/(2*m)) * (theta_t[1:].T @ theta_t[1:])
    J = J + reg
    return J

In [272]:
def lrGradientDescent(theta, X, y, lambda_t):
    m = len(y)
    grad = np.zeros([m,1])
    grad = (1/m) * X.T @ (sigmoid(X @ theta) - y)
    grad[1:] = grad[1:] + (lambda_t / m) * theta[1:]
    return grad

In [273]:
X3 = np.asarray(X_train)
y3 = np.asarray(Y_train)

X3_test = np.asarray(X_test)
y3_test = np.asarray(Y_test)


theta = np.zeros((len(X_train[0]),1))
lmbda = 0.1
J = lrCostFunction(theta, X3, y3, lmbda)
print(J)

[[0.69314718]]


In [274]:
output = opt.fmin_tnc(func = lrCostFunction, x0 = theta.flatten(), fprime = lrGradientDescent, \
                         args = (X3, y3.flatten(), lmbda))
theta = output[0]
print(theta) # theta contains the optimized values

[ 4.84724832e-02 -1.93462554e-02  1.98781610e+01 -3.91508631e+01
 -1.64249736e-01 -8.18023647e-02 -1.75886751e-02 -1.20582376e-01
  1.92387808e-02]


In [275]:
def predict(theta, X):
    probability = sigmoid(X * theta.T)
    return [1 if x >= 0.5 else 0 for x in probability]


theta_min = np.matrix(output[0])
predictions = predict(theta_min, X3_test)
correct = [1 if ((a == 1 and b == 1) or (a == 0 and b == 0)) else 0 for (a, b) in zip(predictions, y3_test)]
accuracy = (sum(map(int, correct)) / len(correct))
print ('accuracy = {0}%'.format(accuracy) )

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

print( accuracy_metric(y3_test, predictions) )



pred = [sigmoid(np.dot(X3_test, theta)) >= 0.5]
np.mean(pred == y3_test.flatten()) * 100

accuracy = 0.713%
71.3


71.3

I think that the predictive performance, an accuracy of ~87%, is quite "reasonable" given that we "only" used the default parameters and didn't do any hyperparameter optimization. 

After we estimated the model perfomance, let us use those last 5,000 test samples to update our model.

In [18]:
#clf = clf.partial_fit(X_test, y_test)

<br>
<br>

# Model Persistence

In the previous section, we successfully trained a model to predict the sentiment of a movie review. Unfortunately, if we'd close this IPython notebook at this point, we'd have to go through the whole learning process again and again if we'd want to make a prediction on "new data."

So, to reuse this model, we could use the [`pickle`](https://docs.python.org/3.5/library/pickle.html) module to "serialize a Python object structure". Or even better, we could use the [`joblib`](https://pypi.python.org/pypi/joblib) library, which handles large NumPy arrays more efficiently.

To install:
conda install -c anaconda joblib

In [21]:
import joblib
import os
if not os.path.exists('./pkl_objects'):
    os.mkdir('./pkl_objects')
    
joblib.dump(vect, './vectorizer.pkl')
joblib.dump(clf, './clf.pkl')

['./clf.pkl']

Using the code above, we "pickled" the `HashingVectorizer` and the `SGDClassifier` so that we can re-use those objects later. However, `pickle` and `joblib` have a known issue with `pickling` objects or functions from a `__main__` block and we'd get an `AttributeError: Can't get attribute [x] on <module '__main__'>` if we'd unpickle it later. Thus, to pickle the `tokenizer` function, we can write it to a file and import it to get the `namespace` "right".

In [22]:
%%writefile tokenizer.py
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    text = [w for w in text.split() if w not in stop]
    tokenized = [porter.stem(w) for w in text]
    return text

Writing tokenizer.py


In [23]:
from tokenizer import tokenizer
joblib.dump(tokenizer, './tokenizer.pkl')

['./tokenizer.pkl']

Now, let us restart this IPython notebook and check if the we can load our serialized objects:

In [24]:
import joblib
tokenizer = joblib.load('./tokenizer.pkl')
vect = joblib.load('./vectorizer.pkl')
clf = joblib.load('./clf.pkl')

After loading the `tokenizer`, `HashingVectorizer`, and the tranined logistic regression model, we can use it to make predictions on new data, which can be useful, for example, if we'd want to embed our classifier into a web application -- a topic for another IPython notebook.

In [25]:
example = ['I did not like this movie']
X = vect.transform(example)
clf.predict(X)

array([0])

In [26]:
example = ['I loved this movie']
X = vect.transform(example)
clf.predict(X)

array([1])